# Daily Fill Aggressiveness

Measure how aggressively weekly orders are getting filled each trading day.

Uses `{TEAM_ID}/targets.csv` and computes:
- `incremental_fill_pct`: percent of the weekly order filled *on that day* (same-day observed)
- `cumulative_fill_pct`: percent of the weekly order filled *by that day* (same-day observed)
- `incremental_fill_pct_settled_t1`: same metric but using next-day `actual_w` as a T+1 settled proxy
- `cumulative_fill_pct_settled_t1`: cumulative T+1 settled proxy

Plots are shown by:
- calendar weekday (Mon-Fri)
- scale day index (0..N)


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from io import StringIO
from IPython.display import display

pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (14, 6)

from QuantConnect import *
from QuantConnect.Research import QuantBook
from config import TEAM_ID

qb = QuantBook()
print('QuantBook initialized')


def read_csv_from_store(key):
    try:
        if not qb.ObjectStore.ContainsKey(key):
            print(f'ObjectStore key not found: {key}')
            return None
        content = qb.ObjectStore.Read(key)
        if not content:
            print(f'Empty ObjectStore key: {key}')
            return None
        return pd.read_csv(StringIO(content))
    except Exception as e:
        print(f'Error reading {key}: {e}')
        return None


## Data Loading — Targets Log

Loads the `targets.csv` log from ObjectStore, which records the daily scheduled and actual weight for each symbol within each rebalance week. The head preview confirms columns like `start_w`, `weekly_target_w`, `scheduled_fraction`, `actual_w`, and `scale_day` are present. All subsequent fill-aggressiveness calculations depend on this DataFrame.

## Fill Aggressiveness Metric Computation

This cell derives four fill-aggressiveness metrics per row: same-day incremental and cumulative fill percentage, and T+1 settled proxies that substitute the next trading day's actual weight for a more conservative estimate. The preview shows the computed columns alongside original target data for the first 15 rows. The two proxy series allow side-by-side comparison to detect systematic same-day versus settlement timing discrepancies.

## Weekday Fill Summary Tables

These two tables summarize fill aggressiveness by calendar weekday at both the portfolio level (weighted by weekly order size) and the symbol level (simple mean). They show what fraction of the weekly order is typically filled on each day and how cumulative fill builds up across the trading week. Comparing same-day versus T+1 settled numbers by weekday highlights whether reported aggressiveness on any given day is partly an artifact of next-day settlement.

## Weekday Fill Aggressiveness Charts

These two side-by-side charts visualize fill aggressiveness by calendar weekday. The left panel shows the average fraction of the weekly order filled on each weekday, comparing same-day versus T+1 settled estimates as grouped bars. The right panel shows the cumulative fill percentage reached by each weekday, **restricted to complete 5-day symbol-weeks** to avoid Simpson's paradox (where changing composition across weekdays can make cumulative averages appear to decrease).

## Scale Day Fill Profile — Table and Charts

This section repeats the fill analysis indexed by scale day (0, 1, 2…) rather than calendar weekday, which is more natural for the strategy's 5-day scaling schedule. The table shows average incremental and cumulative fill percentage for each scale day, and the two charts display the same data as grouped bars and a cumulative line. Front-loaded curves for strong-tier signals and a flatter profile for weak-tier signals would confirm the tier-dependent scaling design is working as intended.

## Earliest Fill Symbol Ranking (Days 0–1)

This table ranks individual symbols by their average fill aggressiveness on the first two scale days, highlighting which names tend to complete their orders earliest in the week. Consistently early fillers are typically more liquid stocks where limit orders at tight offsets clear quickly, while late fillers may require wider limits or more patient execution. Use this table alongside signal tier data to check whether early fills align with the strong-signal design intent.

In [ ]:
df_targets = read_csv_from_store(f'{TEAM_ID}/targets.csv')

if df_targets is None:
    raise ValueError('targets.csv is required. Run a backtest with target-state logging first.')

df_targets['date'] = pd.to_datetime(df_targets['date'])
numeric_cols = ['start_w', 'weekly_target_w', 'scheduled_fraction', 'scheduled_w', 'actual_w', 'scale_day']
for col in numeric_cols:
    if col in df_targets.columns:
        df_targets[col] = pd.to_numeric(df_targets[col], errors='coerce').fillna(0.0)

print(f'target rows: {len(df_targets):,}')
display(df_targets.head())


In [ ]:
df = df_targets.copy()

# Keep canonical sort for week/day calculations.
df = df.sort_values(['week_id', 'symbol', 'date'])

# Weekly order size and remaining amount (same-day observed)
df['weekly_order_abs'] = (df['weekly_target_w'] - df['start_w']).abs()
df['remaining_abs'] = (df['weekly_target_w'] - df['actual_w']).abs()
df['remaining_abs'] = np.minimum(df['remaining_abs'], df['weekly_order_abs'])

# T+1 settled proxy: use next trading day's actual weight WITHIN THE SAME WEEK_ID.
# This avoids cross-cycle contamination on the final day of a rebalance cycle.
df['actual_w_settled_t1'] = (
    df.groupby(['week_id', 'symbol'])['actual_w']
      .shift(-1)
      .fillna(df['actual_w'])
)

# Same-day incremental/cumulative fill.
df['prior_remaining_abs'] = df.groupby(['week_id', 'symbol'])['remaining_abs'].shift(1)
df['prior_remaining_abs'] = df['prior_remaining_abs'].fillna(df['weekly_order_abs'])
df['incremental_fill_abs'] = (df['prior_remaining_abs'] - df['remaining_abs']).clip(lower=0)

df['incremental_fill_pct'] = np.where(
    df['weekly_order_abs'] > 1e-10,
    df['incremental_fill_abs'] / df['weekly_order_abs'],
    0.0
)

df['cumulative_fill_pct'] = np.where(
    df['weekly_order_abs'] > 1e-10,
    (df['weekly_order_abs'] - df['remaining_abs']) / df['weekly_order_abs'],
    1.0
)

# T+1 settled incremental/cumulative fill.
df['remaining_abs_settled_t1'] = (df['weekly_target_w'] - df['actual_w_settled_t1']).abs()
df['remaining_abs_settled_t1'] = np.minimum(df['remaining_abs_settled_t1'], df['weekly_order_abs'])

df['prior_remaining_abs_settled_t1'] = df.groupby(['week_id', 'symbol'])['remaining_abs_settled_t1'].shift(1)
df['prior_remaining_abs_settled_t1'] = df['prior_remaining_abs_settled_t1'].fillna(df['weekly_order_abs'])
df['incremental_fill_abs_settled_t1'] = (
    df['prior_remaining_abs_settled_t1'] - df['remaining_abs_settled_t1']
).clip(lower=0)

df['incremental_fill_pct_settled_t1'] = np.where(
    df['weekly_order_abs'] > 1e-10,
    df['incremental_fill_abs_settled_t1'] / df['weekly_order_abs'],
    0.0
)

df['cumulative_fill_pct_settled_t1'] = np.where(
    df['weekly_order_abs'] > 1e-10,
    (df['weekly_order_abs'] - df['remaining_abs_settled_t1']) / df['weekly_order_abs'],
    1.0
)

# Clip to [0,1] for stable plotting.
for col in [
    'incremental_fill_pct',
    'cumulative_fill_pct',
    'incremental_fill_pct_settled_t1',
    'cumulative_fill_pct_settled_t1'
]:
    df[col] = df[col].clip(0, 1)

df['weekday'] = df['date'].dt.day_name()
weekday_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']
df['weekday'] = pd.Categorical(df['weekday'], categories=weekday_order, ordered=True)

df = df[df['weekly_order_abs'] > 1e-10].copy()

display(
    df[[
        'date', 'week_id', 'symbol', 'scale_day',
        'incremental_fill_pct', 'cumulative_fill_pct',
        'incremental_fill_pct_settled_t1', 'cumulative_fill_pct_settled_t1'
    ]].head(15)
)


In [ ]:
# Portfolio-level daily fill aggressiveness (weighted by order size)
by_date = (
    df.groupby('date', as_index=False)
      .agg(
          incremental_fill_abs=('incremental_fill_abs', 'sum'),
          incremental_fill_abs_settled_t1=('incremental_fill_abs_settled_t1', 'sum'),
          weekly_order_abs=('weekly_order_abs', 'sum')
      )
)
by_date['incremental_fill_pct'] = np.where(
    by_date['weekly_order_abs'] > 1e-10,
    by_date['incremental_fill_abs'] / by_date['weekly_order_abs'],
    0.0
)
by_date['incremental_fill_pct_settled_t1'] = np.where(
    by_date['weekly_order_abs'] > 1e-10,
    by_date['incremental_fill_abs_settled_t1'] / by_date['weekly_order_abs'],
    0.0
)
by_date['weekday'] = pd.Categorical(by_date['date'].dt.day_name(), categories=weekday_order, ordered=True)

weekday_weighted = (
    by_date.groupby('weekday', as_index=False)
           .agg(
               avg_incremental_fill_pct=('incremental_fill_pct', 'mean'),
               avg_incremental_fill_pct_settled_t1=('incremental_fill_pct_settled_t1', 'mean')
           )
)

# Cumulative fill by weekday: only use complete 5-day symbol-weeks to avoid
# Simpson's paradox (changing composition across weekdays).
weekday_counts = df.groupby(['week_id', 'symbol'])['weekday'].nunique()
complete_pairs = weekday_counts[weekday_counts == 5].reset_index()[['week_id', 'symbol']]
df_complete = df.merge(complete_pairs, on=['week_id', 'symbol'], how='inner')

weekday_cumulative = (
    df_complete.groupby('weekday', as_index=False)
               .agg(
                   avg_cumulative_fill_pct=('cumulative_fill_pct', 'mean'),
                   avg_cumulative_fill_pct_settled_t1=('cumulative_fill_pct_settled_t1', 'mean'),
                   n_obs=('cumulative_fill_pct', 'count')
               )
)

weekday_symbol_mean = (
    df.groupby('weekday', as_index=False)
      .agg(
          avg_incremental_fill_pct=('incremental_fill_pct', 'mean'),
          avg_incremental_fill_pct_settled_t1=('incremental_fill_pct_settled_t1', 'mean'),
          med_incremental_fill_pct=('incremental_fill_pct', 'median'),
          avg_cumulative_fill_pct=('cumulative_fill_pct', 'mean'),
          avg_cumulative_fill_pct_settled_t1=('cumulative_fill_pct_settled_t1', 'mean')
      )
)

print(f'Complete 5-day symbol-weeks: {len(complete_pairs):,} '
      f'(of {len(weekday_counts):,} total)')
display(weekday_weighted)
display(weekday_cumulative)


In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(16, 5))

weekday_plot = weekday_weighted.copy()
weekday_plot['same_day_pct'] = 100 * weekday_plot['avg_incremental_fill_pct']
weekday_plot['settled_t1_pct'] = 100 * weekday_plot['avg_incremental_fill_pct_settled_t1']
weekday_long = weekday_plot.melt(
    id_vars='weekday',
    value_vars=['same_day_pct', 'settled_t1_pct'],
    var_name='series',
    value_name='value'
)
weekday_long['series'] = weekday_long['series'].map({
    'same_day_pct': 'Same-day observed',
    'settled_t1_pct': 'T+1 settled proxy'
})

sns.barplot(
    data=weekday_long,
    x='weekday',
    y='value',
    hue='series',
    ax=axes[0]
)
axes[0].set_title('Weighted Average % Filled Per Day (Calendar Weekday)')
axes[0].set_ylabel('% of weekly order filled on that day')
axes[0].set_xlabel('Weekday')
axes[0].grid(axis='y', alpha=0.3)

# Use complete 5-day weeks only so cumulative is guaranteed monotonic.
axes[1].plot(
    weekday_cumulative['weekday'],
    100 * weekday_cumulative['avg_cumulative_fill_pct'],
    marker='o',
    color='#d62728',
    label='Same-day observed'
)
axes[1].plot(
    weekday_cumulative['weekday'],
    100 * weekday_cumulative['avg_cumulative_fill_pct_settled_t1'],
    marker='o',
    color='#1f77b4',
    label='T+1 settled proxy'
)
axes[1].set_title('Average Cumulative Fill % By Weekday (Complete Weeks Only)')
axes[1].set_ylabel('Cumulative fill %')
axes[1].set_xlabel('Weekday')
axes[1].grid(alpha=0.3)
axes[1].legend()

plt.tight_layout()
plt.show()


In [ ]:
scale_day_profile = (
    df.groupby('scale_day', as_index=False)
      .agg(
          avg_incremental_fill_pct=('incremental_fill_pct', 'mean'),
          avg_incremental_fill_pct_settled_t1=('incremental_fill_pct_settled_t1', 'mean'),
          med_incremental_fill_pct=('incremental_fill_pct', 'median'),
          avg_cumulative_fill_pct=('cumulative_fill_pct', 'mean'),
          avg_cumulative_fill_pct_settled_t1=('cumulative_fill_pct_settled_t1', 'mean')
      )
      .sort_values('scale_day')
)

display(scale_day_profile)

fig, axes = plt.subplots(1, 2, figsize=(16, 5))

x = scale_day_profile['scale_day'].to_numpy()
width = 0.38
axes[0].bar(x - width/2, 100 * scale_day_profile['avg_incremental_fill_pct'], width=width, color='#2ca02c', label='Same-day observed')
axes[0].bar(x + width/2, 100 * scale_day_profile['avg_incremental_fill_pct_settled_t1'], width=width, color='#1f77b4', label='T+1 settled proxy')
axes[0].set_title('Average % Filled Per Scale Day')
axes[0].set_xlabel('Scale day')
axes[0].set_ylabel('% of weekly order filled that day')
axes[0].set_xticks(x)
axes[0].legend()
axes[0].grid(axis='y', alpha=0.3)

axes[1].plot(scale_day_profile['scale_day'], 100 * scale_day_profile['avg_cumulative_fill_pct'], marker='o', color='#9467bd', label='Same-day observed')
axes[1].plot(scale_day_profile['scale_day'], 100 * scale_day_profile['avg_cumulative_fill_pct_settled_t1'], marker='o', color='#ff7f0e', label='T+1 settled proxy')
axes[1].set_title('Average Cumulative Fill % by Scale Day')
axes[1].set_xlabel('Scale day')
axes[1].set_ylabel('Cumulative fill %')
axes[1].legend()
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.show()


In [ ]:
# Optional: symbol-level ranking of aggressiveness on day 0 and day 1
day_early = df[df['scale_day'].isin([0, 1])]
symbol_early = (
    day_early.groupby('symbol', as_index=False)
             .agg(avg_fill_day0_1=('incremental_fill_pct', 'mean'),
                  observations=('incremental_fill_pct', 'count'))
             .sort_values('avg_fill_day0_1', ascending=False)
)

print('Most aggressive early fills (day 0-1):')
display(symbol_early.head(15))


## Cycle Carryover and Convergence (By Signal Strength)

This section tracks the per-cycle completion dynamic you described:
- cycle start gap = `|weekly_target_w - start_w|`
- cycle end gap = `|weekly_target_w - week_end_actual_w|`
- cycle fill % = `(start gap - end gap) / start gap`
- carryover % = `1 - cycle fill %`

Signal strength/tier is attached from `order_events.csv` order tags when available, with fallback to `signals.csv`.


In [ ]:
# Build weekly diagnostics from the per-day target-state table.
weekly_diag = (
    df.sort_values(['week_id', 'symbol', 'date'])
      .groupby(['week_id', 'symbol'], as_index=False)
      .agg(
          week_start=('date', 'min'),
          week_end=('date', 'max'),
          start_w=('start_w', 'first'),
          weekly_target_w=('weekly_target_w', 'first'),
          week_end_actual_w=('actual_w', 'last'),
          max_scale_day=('scale_day', 'max')
      )
)

EPS = 1e-10
SCALE_DAYS_EXPECTED = 5
TARGET_STABILITY_TOL = 0.0025  # 25 bps target-change tolerance for episode stitching

weekly_diag['gap_start_abs'] = (weekly_diag['weekly_target_w'] - weekly_diag['start_w']).abs()
weekly_diag['gap_end_abs'] = (weekly_diag['weekly_target_w'] - weekly_diag['week_end_actual_w']).abs()
weekly_diag['filled_abs'] = (weekly_diag['gap_start_abs'] - weekly_diag['gap_end_abs']).clip(lower=0)

weekly_diag['cycle_fill_pct'] = np.where(
    weekly_diag['gap_start_abs'] > EPS,
    weekly_diag['filled_abs'] / weekly_diag['gap_start_abs'],
    1.0
).clip(0, 1)
weekly_diag['carryover_pct'] = (1.0 - weekly_diag['cycle_fill_pct']).clip(0, 1)

# Only compare complete 5-scale-day cycles for fairness.
weekly_diag['full_scale_week'] = weekly_diag['max_scale_day'] >= (SCALE_DAYS_EXPECTED - 1)

# Intent classification for cleaner comparisons.
start_abs = weekly_diag['start_w'].abs()
target_abs = weekly_diag['weekly_target_w'].abs()
flip = (
    (start_abs > EPS) & (target_abs > EPS) &
    (np.sign(weekly_diag['start_w']) != np.sign(weekly_diag['weekly_target_w']))
)
exit_to_zero = (start_abs > EPS) & (target_abs <= EPS)
entry_new = (start_abs <= EPS) & (target_abs > EPS)
increase = (~flip) & (~entry_new) & (target_abs > start_abs + EPS)
decrease = (~flip) & (~exit_to_zero) & (target_abs + EPS < start_abs)

weekly_diag['intent_bucket'] = np.select(
    [flip, exit_to_zero, entry_new, increase, decrease],
    ['flip', 'exit_to_zero', 'entry_new', 'increase', 'decrease'],
    default='flat_or_small_change'
)


def parse_order_tag(tag):
    parsed = {}
    if pd.isna(tag):
        return parsed
    for part in str(tag).split(';'):
        if '=' not in part:
            continue
        k, v = part.split('=', 1)
        parsed[k.strip()] = v.strip()
    return parsed


def tier_from_strength(x):
    if pd.isna(x):
        return np.nan
    if x >= 0.70:
        return 'strong'
    if x >= 0.30:
        return 'moderate'
    return 'weak'


def choose_tier(series):
    s = series.dropna().astype(str)
    s = s[s != '']
    if s.empty:
        return np.nan
    non_exit = s[s != 'exit']
    if not non_exit.empty:
        return non_exit.mode().iloc[0]
    if (s == 'exit').any():
        return 'exit'
    return s.mode().iloc[0]


# 1) Primary tier/strength mapping from order-event tags.
week_signal_order = None
df_order_events = read_csv_from_store(f'{TEAM_ID}/order_events.csv')
if df_order_events is not None and 'tag' in df_order_events.columns:
    oe = df_order_events.copy()
    oe['symbol'] = oe['symbol'].astype(str)

    if 'date' in oe.columns:
        oe['date'] = pd.to_datetime(oe['date'], errors='coerce')

    tag_map = oe['tag'].apply(parse_order_tag)
    oe['week_id'] = tag_map.apply(lambda x: x.get('week_id', ''))
    oe['signal_tier'] = tag_map.apply(lambda x: x.get('tier', ''))
    oe['signal_strength'] = pd.to_numeric(
        tag_map.apply(lambda x: x.get('signal', np.nan)),
        errors='coerce'
    ).abs()

    oe = oe[(oe['week_id'].notna()) & (oe['week_id'] != '')]

    # Collapse repeated status rows per order id, keep last event row.
    if 'order_id' in oe.columns:
        sort_cols = ['order_id'] + (['date'] if 'date' in oe.columns else [])
        oe = oe.sort_values(sort_cols).drop_duplicates(subset=['order_id'], keep='last')

    if not oe.empty:
        week_signal_order = (
            oe.groupby(['week_id', 'symbol'], as_index=False)
              .agg(
                  signal_strength=('signal_strength', 'median'),
                  signal_tier=('signal_tier', choose_tier)
              )
        )

# 2) Fallback mapping from signals.csv magnitude.
week_signal_fallback = None
df_signals = read_csv_from_store(f'{TEAM_ID}/signals.csv')
if df_signals is not None and {'date', 'symbol', 'magnitude'}.issubset(df_signals.columns):
    sig = df_signals.copy()
    sig['date'] = pd.to_datetime(sig['date'], errors='coerce')
    sig = sig.dropna(subset=['date'])
    sig['week_id'] = sig['date'].dt.strftime('%Y-%m-%d')
    sig['symbol'] = sig['symbol'].astype(str)
    sig['signal_strength'] = pd.to_numeric(sig['magnitude'], errors='coerce').abs()

    week_signal_fallback = (
        sig.groupby(['week_id', 'symbol'], as_index=False)
           .agg(signal_strength=('signal_strength', 'max'))
    )
    week_signal_fallback['signal_tier'] = week_signal_fallback['signal_strength'].apply(tier_from_strength)

# Merge mappings with order-event precedence.
if week_signal_order is None:
    week_signal_order = pd.DataFrame(columns=['week_id', 'symbol', 'signal_strength', 'signal_tier'])
if week_signal_fallback is None:
    week_signal_fallback = pd.DataFrame(columns=['week_id', 'symbol', 'signal_strength', 'signal_tier'])

week_signal = week_signal_order.merge(
    week_signal_fallback,
    on=['week_id', 'symbol'],
    how='outer',
    suffixes=('_order', '_fallback')
)

week_signal['signal_strength'] = week_signal['signal_strength_order'].fillna(week_signal['signal_strength_fallback'])
week_signal['signal_tier'] = week_signal['signal_tier_order'].replace('', np.nan)
week_signal['signal_tier'] = week_signal['signal_tier'].fillna(week_signal['signal_tier_fallback'])
week_signal['signal_tier'] = week_signal['signal_tier'].fillna(week_signal['signal_strength'].apply(tier_from_strength))
week_signal = week_signal[['week_id', 'symbol', 'signal_strength', 'signal_tier']].drop_duplicates(['week_id', 'symbol'], keep='last')

weekly_diag = weekly_diag.merge(week_signal, on=['week_id', 'symbol'], how='left')
weekly_diag['signal_tier_raw'] = weekly_diag['signal_tier'].fillna('unknown')
weekly_diag['signal_tier'] = weekly_diag['signal_tier_raw']

# Cleanup unknowns:
# Unknowns are typically pure exit weeks (target=0) with no active signal emitted/tagged.
mask_unknown_exit = (
    (weekly_diag['signal_tier'] == 'unknown') &
    (weekly_diag['intent_bucket'] == 'exit_to_zero')
)
weekly_diag.loc[mask_unknown_exit, 'signal_tier'] = 'exit'
weekly_diag.loc[mask_unknown_exit, 'signal_strength'] = weekly_diag.loc[mask_unknown_exit, 'signal_strength'].fillna(0.0)

# Track prior non-exit tier so exits can be attributed to the tier being unwound.
weekly_diag = weekly_diag.sort_values(['symbol', 'week_start']).reset_index(drop=True)
weekly_diag['prior_non_exit_tier'] = (
    weekly_diag['signal_tier']
      .where(weekly_diag['signal_tier'].isin(['strong', 'moderate', 'weak']))
      .groupby(weekly_diag['symbol'])
      .ffill()
)
weekly_diag['exit_from_tier'] = np.where(
    weekly_diag['signal_tier'] == 'exit',
    weekly_diag['prior_non_exit_tier'],
    np.nan
)

# Effective tier for optional attribution of exits to prior tier.
weekly_diag['signal_tier_effective'] = weekly_diag['signal_tier']
mask_exit_with_parent = (
    (weekly_diag['signal_tier'] == 'exit') &
    (weekly_diag['exit_from_tier'].notna())
)
weekly_diag.loc[mask_exit_with_parent, 'signal_tier_effective'] = weekly_diag.loc[mask_exit_with_parent, 'exit_from_tier']

# Symbol-week sequence metrics for reset diagnostics.
weekly_diag['prev_target_w'] = weekly_diag.groupby('symbol')['weekly_target_w'].shift(1)
weekly_diag['target_shift_abs'] = (weekly_diag['weekly_target_w'] - weekly_diag['prev_target_w']).abs()
weekly_diag['prev_carryover_pct'] = weekly_diag.groupby('symbol')['carryover_pct'].shift(1)

weekly_diag['same_target_prev'] = weekly_diag['target_shift_abs'] <= TARGET_STABILITY_TOL
weekly_diag['prev_tier'] = weekly_diag.groupby('symbol')['signal_tier'].shift(1)
weekly_diag['same_tier_prev'] = weekly_diag['signal_tier'].eq(weekly_diag['prev_tier'])

weekly_diag['new_episode'] = (
    weekly_diag['prev_target_w'].isna() |
    (~weekly_diag['same_target_prev']) |
    (~weekly_diag['same_tier_prev'])
)
weekly_diag['episode_id'] = weekly_diag.groupby('symbol')['new_episode'].cumsum()
weekly_diag['week_in_episode'] = weekly_diag.groupby(['symbol', 'episode_id']).cumcount() + 1

weekly_diag['target_reset_with_carryover'] = (
    (weekly_diag['target_shift_abs'] > TARGET_STABILITY_TOL) &
    (weekly_diag['prev_carryover_pct'].fillna(0) > 0.20)
).astype(int)

# Attach cycle-level diagnostics back to daily rows for tier-aware day curves.
for col in [
    'signal_strength', 'signal_tier', 'signal_tier_raw', 'signal_tier_effective',
    'exit_from_tier', 'full_scale_week', 'intent_bucket'
]:
    if col in df.columns:
        df.drop(columns=[col], inplace=True)

df = df.merge(
    weekly_diag[[
        'week_id', 'symbol', 'signal_strength', 'signal_tier', 'signal_tier_raw',
        'signal_tier_effective', 'exit_from_tier', 'full_scale_week', 'intent_bucket'
    ]],
    on=['week_id', 'symbol'],
    how='left'
)
df['signal_tier'] = df['signal_tier'].fillna('unknown')

tier_counts_raw = weekly_diag.groupby('signal_tier_raw', as_index=False).size().sort_values('size', ascending=False)
tier_counts_clean = weekly_diag.groupby('signal_tier', as_index=False).size().sort_values('size', ascending=False)
exit_attribution = (
    weekly_diag[weekly_diag['signal_tier'] == 'exit']
    .groupby('exit_from_tier', as_index=False)
    .size()
    .sort_values('size', ascending=False)
)

print(f'weekly rows: {len(weekly_diag):,}')
print('tier counts (raw):')
display(tier_counts_raw)
print('tier counts (cleaned):')
display(tier_counts_clean)
print('exit attribution (prior tier):')
display(exit_attribution)

display(
    weekly_diag[[
        'week_id', 'symbol', 'signal_tier_raw', 'signal_tier', 'signal_tier_effective',
        'exit_from_tier', 'signal_strength', 'intent_bucket', 'full_scale_week',
        'start_w', 'weekly_target_w', 'gap_start_abs', 'gap_end_abs',
        'cycle_fill_pct', 'carryover_pct', 'target_shift_abs',
        'target_reset_with_carryover', 'week_in_episode'
    ]].head(20)
)


## Signal-Tier Carryover Dashboards

These views monitor whether positions are converging to target or staying underweight due to carryover + target resets.


In [ ]:
tier_order = ['strong', 'moderate', 'weak']


def summarize_cycles(frame, label):
    rows = []
    for tier in tier_order:
        g = frame[frame['signal_tier'] == tier]
        if g.empty:
            continue
        gap_sum = g['gap_start_abs'].sum()
        filled_sum = g['filled_abs'].sum()
        weighted_fill = (filled_sum / gap_sum) if gap_sum > 1e-10 else np.nan
        rows.append({
            'view': label,
            'signal_tier': tier,
            'weeks': int(len(g)),
            'avg_start_gap_pct': 100 * g['gap_start_abs'].mean(),
            'avg_cycle_fill_pct': 100 * g['cycle_fill_pct'].mean(),
            'weighted_cycle_fill_pct': 100 * weighted_fill if pd.notna(weighted_fill) else np.nan,
            'avg_carryover_pct': 100 * g['carryover_pct'].mean(),
            'weighted_carryover_pct': 100 * (1 - weighted_fill) if pd.notna(weighted_fill) else np.nan,
            'reset_while_unfilled_rate': 100 * g['target_reset_with_carryover'].mean()
        })
    return pd.DataFrame(rows)


# View A: all non-exit tiered cycles on complete scale windows.
view_a = weekly_diag[
    weekly_diag['full_scale_week'] &
    weekly_diag['signal_tier'].isin(tier_order)
].copy()

# View B: entry/add cycles only (closest to "underweight relative to target" concern).
view_b = weekly_diag[
    weekly_diag['full_scale_week'] &
    weekly_diag['signal_tier'].isin(tier_order) &
    weekly_diag['intent_bucket'].isin(['entry_new', 'increase'])
].copy()

cycle_summary = pd.concat([
    summarize_cycles(view_a, 'All Non-Exit Cycles (Full Weeks)'),
    summarize_cycles(view_b, 'Entry/Add Cycles Only (Full Weeks)')
], ignore_index=True)

display(cycle_summary)

# Day-by-day cumulative fill profile by tier for entry/add cycles.
df_plot = df[
    df['full_scale_week'].fillna(False) &
    df['signal_tier'].isin(tier_order) &
    df['intent_bucket'].isin(['entry_new', 'increase'])
].copy()

scale_tier_profile = (
    df_plot.groupby(['signal_tier', 'scale_day'], as_index=False)
          .agg(
              avg_incremental_fill_pct=('incremental_fill_pct', 'mean'),
              avg_cumulative_fill_pct=('cumulative_fill_pct', 'mean'),
              observations=('symbol', 'count')
          )
          .sort_values(['signal_tier', 'scale_day'])
)

display(scale_tier_profile)

# Episode-level convergence profile for stable-target episodes.
episode_len = (
    weekly_diag.groupby(['symbol', 'episode_id'], as_index=False)
               .size()
               .rename(columns={'size': 'episode_len'})
)
weekly_episode = weekly_diag.merge(episode_len, on=['symbol', 'episode_id'], how='left')
weekly_episode = weekly_episode[
    (weekly_episode['episode_len'] >= 2) &
    weekly_episode['signal_tier'].isin(tier_order) &
    weekly_episode['intent_bucket'].isin(['entry_new', 'increase'])
].copy()

episode_profile = (
    weekly_episode.groupby(['signal_tier', 'week_in_episode'], as_index=False)
                 .agg(
                     avg_gap_start_abs=('gap_start_abs', 'mean'),
                     avg_gap_end_abs=('gap_end_abs', 'mean'),
                     avg_cycle_fill_pct=('cycle_fill_pct', 'mean'),
                     observations=('week_id', 'count')
                 )
                 .sort_values(['signal_tier', 'week_in_episode'])
)

episode_profile['avg_gap_start_pct'] = 100 * episode_profile['avg_gap_start_abs']
episode_profile['avg_gap_end_pct'] = 100 * episode_profile['avg_gap_end_abs']
episode_profile['avg_carryover_pct'] = np.where(
    episode_profile['avg_gap_start_abs'] > 1e-10,
    100 * episode_profile['avg_gap_end_abs'] / episode_profile['avg_gap_start_abs'],
    np.nan
)

display(episode_profile)

fig, axes = plt.subplots(2, 2, figsize=(18, 10))

# 1) Cycle completion by tier (entry/add view).
bar_data = cycle_summary[cycle_summary['view'] == 'Entry/Add Cycles Only (Full Weeks)'].copy()
bar_data['signal_tier'] = pd.Categorical(bar_data['signal_tier'], categories=tier_order, ordered=True)
bar_data = bar_data.sort_values('signal_tier')

x = np.arange(len(bar_data))
width = 0.38
axes[0, 0].bar(x - width/2, bar_data['avg_cycle_fill_pct'], width=width, color='#2ca02c', label='Unweighted avg')
axes[0, 0].bar(x + width/2, bar_data['weighted_cycle_fill_pct'], width=width, color='#1f77b4', label='Gap-weighted')
axes[0, 0].set_xticks(x)
axes[0, 0].set_xticklabels(bar_data['signal_tier'].astype(str))
axes[0, 0].set_title('Average 5-Day Cycle Fill % by Tier (Entry/Add)')
axes[0, 0].set_ylabel('Cycle fill %')
axes[0, 0].grid(axis='y', alpha=0.3)
axes[0, 0].legend()

# 2) Cumulative fill by scale day and tier.
for tier in tier_order:
    sub = scale_tier_profile[scale_tier_profile['signal_tier'] == tier]
    if sub.empty:
        continue
    axes[0, 1].plot(sub['scale_day'], 100 * sub['avg_cumulative_fill_pct'], marker='o', label=tier)
axes[0, 1].set_title('Cumulative Fill % by Scale Day and Tier (Entry/Add)')
axes[0, 1].set_xlabel('Scale day')
axes[0, 1].set_ylabel('Cumulative fill %')
axes[0, 1].grid(alpha=0.3)
axes[0, 1].legend(loc='best')

# 3) Start-gap decay across stable-target episodes.
for tier in tier_order:
    sub = episode_profile[episode_profile['signal_tier'] == tier]
    if sub.empty:
        continue
    axes[1, 0].plot(sub['week_in_episode'], sub['avg_gap_start_pct'], marker='o', label=tier)
axes[1, 0].set_title('Start Gap % by Week-in-Episode (Stable Target)')
axes[1, 0].set_xlabel('Week in episode')
axes[1, 0].set_ylabel('Start gap (% NAV)')
axes[1, 0].grid(alpha=0.3)
axes[1, 0].legend()

# 4) Carryover % across stable-target episodes.
for tier in tier_order:
    sub = episode_profile[episode_profile['signal_tier'] == tier]
    if sub.empty:
        continue
    axes[1, 1].plot(sub['week_in_episode'], sub['avg_carryover_pct'], marker='o', label=tier)
axes[1, 1].set_title('Carryover % by Week-in-Episode (Stable Target)')
axes[1, 1].set_xlabel('Week in episode')
axes[1, 1].set_ylabel('Carryover %')
axes[1, 1].grid(alpha=0.3)
axes[1, 1].legend()

plt.tight_layout()
plt.show()


## Example Convergence Table (Closest to Your Weak-Signal Scenario)

This auto-selects a long stable-target episode (weak tier preferred) and prints week-by-week convergence.


In [ ]:
weekly_episode = weekly_diag.copy()

episode_len = (
    weekly_episode.groupby(['symbol', 'episode_id'], as_index=False)
                 .size()
                 .rename(columns={'size': 'episode_len'})
)
weekly_episode = weekly_episode.merge(episode_len, on=['symbol', 'episode_id'], how='left')

candidates = weekly_episode[(weekly_episode['signal_tier'] == 'weak') & (weekly_episode['episode_len'] >= 3)].copy()
if candidates.empty:
    candidates = weekly_episode[weekly_episode['episode_len'] >= 3].copy()

if candidates.empty:
    print('No episode with length >= 3 found; showing longest available episode.')
    longest = (
        weekly_episode.groupby(['symbol', 'episode_id'], as_index=False)
                     .size()
                     .sort_values('size', ascending=False)
                     .head(1)
    )
else:
    longest = (
        candidates.groupby(['symbol', 'episode_id'], as_index=False)
                  .size()
                  .sort_values(['size', 'symbol'], ascending=[False, True])
                  .head(1)
    )

if longest.empty:
    print('No weekly episodes available.')
else:
    sel_symbol = longest.iloc[0]['symbol']
    sel_episode = longest.iloc[0]['episode_id']

    ex = (
        weekly_episode[
            (weekly_episode['symbol'] == sel_symbol) &
            (weekly_episode['episode_id'] == sel_episode)
        ]
        .sort_values('week_start')
        .copy()
    )

    ex['start_w_pct'] = 100 * ex['start_w']
    ex['weekly_target_w_pct'] = 100 * ex['weekly_target_w']
    ex['gap_start_pct'] = 100 * ex['gap_start_abs']
    ex['filled_this_cycle_pct_of_gap'] = 100 * ex['cycle_fill_pct']
    ex['week_end_actual_w_pct'] = 100 * ex['week_end_actual_w']

    print(f'Example symbol: {sel_symbol} | episode_id: {sel_episode} | tier: {ex.iloc[0]["signal_tier"]}')

    display(
        ex[[
            'week_id',
            'week_in_episode',
            'start_w_pct',
            'weekly_target_w_pct',
            'gap_start_pct',
            'filled_this_cycle_pct_of_gap',
            'week_end_actual_w_pct'
        ]]
        .rename(columns={
            'week_id': 'Week',
            'week_in_episode': 'Week#',
            'start_w_pct': 'start_w (%)',
            'weekly_target_w_pct': 'weekly_target_w (%)',
            'gap_start_pct': 'Gap (%)',
            'filled_this_cycle_pct_of_gap': '% gap filled this cycle',
            'week_end_actual_w_pct': 'actual_w achieved (%)'
        })
    )
